## RSA Testbench


This notebook is to test the implementation of a RSA decryption running on the programmable logic. 

In [1]:
from pynq import Overlay
from pynq import MMIO
import numpy as np
import struct
import binascii
import math
import cmath
import random
import sympy
import matplotlib.pyplot as plt
import time



In [2]:
ol=Overlay('./opt2_demo.bit') #Change name of bitstream as required

In [3]:
rsa_ip=MMIO(0x40000000,10000) #Change base address as required

In [4]:
half_bit = 128
NUM_SAMPLES = 5
def gen_p_q(bit):
    primes = [i for i in range(1 << (bit-2), min((1 << (bit - 2)) + 20000, (1 << (bit-1)))) if sympy.isprime(i)]
    #print(primes)
    while(True):
        p = random.choice(primes)
        if(sympy.isprime(2*p+1)):
            break
        
    while(True):
        q = random.choice(primes)
        if(sympy.isprime(2*q+1) and q != p):
            break
        
    return 2*p+1, 2*q+1


def gen_d(e, p, q):
    d = pow(e, -1, (p-1) * (q-1))
    return d

p, q = gen_p_q(half_bit)
e = 65537
d = gen_d(e, p, q)
n = p * q

In [5]:
# Helper function to split a large integer into 32-bit chunks
def split_256bit(value):
    chunks = [(value >> (32 * i)) & 0xFFFFFFFF for i in range(8)]
    return chunks

def combine_256bit(chunks):
    return sum((chunk << (32 * i)) for i, chunk in enumerate(chunks))

for i in range(NUM_SAMPLES):
    #Computing golden output
    gold_x = random.randint(1, (p-1) * (q-1) - 1)
    
    #Generating random inputs
    y=pow(gold_x, e, n)
    x2 = pow(y, d, n)
    #print(y)
    #print(d)
    #print(n)
    #print(x2)
    
    #Converting input to bytes to be sent to FPGA
    y_chunks = split_256bit(y)
    d_chunks = split_256bit(d)
    n_chunks = split_256bit(n)

    
 
    #Writing values to the FPGA
    n_offsets = [0x34, 0x38, 0x3C, 0x40, 0x44, 0x48, 0x4C, 0x50]
    d_offsets = [0x10, 0x14, 0x18, 0x1C, 0x20, 0x24, 0x28, 0x2C]  
    y_offsets = [0x58, 0x5c, 0x60, 0x64, 0x68, 0x6C, 0x70, 0x74] 
    
    for j in range(8):
        rsa_ip.write(n_offsets[j], n_chunks[j])
        rsa_ip.write(d_offsets[j], d_chunks[j])
        rsa_ip.write(y_offsets[j], y_chunks[j])
        
    #Starting and stopping the IP (Don't change this)
    rsa_ip.write(0x00,1)
    rsa_ip.write(0x00,0)
    
    #Reading from IP
    #x_offsets = [0x98, 0x94, 0x90, 0x8C, 0x88, 0x84, 0x80, 0x7C]
    time.sleep(1)
    x_offsets = [0x7C, 0x80, 0x84, 0x88, 0x8C, 0x90, 0x94, 0x98]
    x_chunks = [rsa_ip.read(offset) for offset in x_offsets]
    x = combine_256bit(x_chunks)

    
    # Displaying the result
    print(f"Result from FPGA: {x}")
    print(f"Golden x: {gold_x}")
    
    
    # Comparing with golden output
    # x (decrypted from FPGA) should match the original gold_x
    if x == gold_x:
        print(f"Test {i + 1}: PASS - FPGA output matches golden output")
    else:
        print(f"Test {i + 1}: FAIL - FPGA output does not match golden output")

Result from FPGA: 1568778431203087221099399762546626605421697598282153037443198053731821249099
Golden x: 1568778431203087221099399762546626605421697598282153037443198053731821249099
Test 1: PASS - FPGA output matches golden output
Result from FPGA: 2904960047143273537652411316718990622592179519742551013914581787794664690576
Golden x: 2904960047143273537652411316718990622592179519742551013914581787794664690576
Test 2: PASS - FPGA output matches golden output
Result from FPGA: 230794932791908138464634396270271154138079502439180112723521049738852512730
Golden x: 230794932791908138464634396270271154138079502439180112723521049738852512730
Test 3: PASS - FPGA output matches golden output
Result from FPGA: 2802673193063888591133985566907465721208309368600998209845582833219575325951
Golden x: 2802673193063888591133985566907465721208309368600998209845582833219575325951
Test 4: PASS - FPGA output matches golden output
Result from FPGA: 118369314380070978579554695162133974393381994030787348807259